<h4>Cleaning Web-scraped Auckland CBD Business Directory Data</h4>

<h5>Import Dependencies</h5>

In [1]:
!pip install w3lib
!pip install selenium
!pip install geocoder
!pip install geopy
!pip install geopy
!pip install folium

import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
import requests
import csv
import pprint as pp
import time
import re
import urllib.parse

from w3lib.url import url_query_parameter
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

# from selenium.common.exceptions import IndexError

from bs4 import BeautifulSoup as bs  # Beautiful Soup v4

# Only use when full dataframe needs to be viewed
# pd.set_option("display.max_rows", None, "display.max_columns", None)

import geocoder  # import geocoder

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium  # plotting library

print('Libraries imported.')

Libraries imported.


<h5>Extract Contents of Excel File into DataFrame</h5>

In [2]:
akl_cbd_biz_extract = pd.read_excel("web_scraper.xlsx")

akl_cbd_biz = akl_cbd_biz_extract.replace(np.nan, '', regex=True)

k = akl_cbd_biz.shape[0]

akl_cbd_biz.head()

,Name,Main Category,Sub Category 1,Sub Category 2,Address Line 1,Address Line 2,Precinct,Post Code,Locality,Phone,Web URL
0,100 Fix,Retail,Electronics and technology,,,19 Shortland Street,\n\nFederal & Elliott \n\n,1010,Auckland,,
1,109 Doctors,Services,Health and wellbeing,Doctors,Level 9,109 Queen Street,\n\nQueen Street \n\n,,Auckland,(09)3660109,
2,16 Tun,Hospitality,Bars and clubs,Restaurants,,10-26 Jellicoe Street,\n\nViaduct & Waterfront \n\n,,Auckland,09 368 7712,www.16tun.com
3,1Above Ltd,Services,Health and wellbeing,,Old Sofrana House Level 1,18-Customs Street East,\n\nViaduct & Waterfront \n\n,,Auckland,09 379 5708,www.fly1above.com
4,2 Degrees,Retail,Electronics and technology,,Civic Centre,8 Wellesley Street,\n\nFederal & Elliott \n\n,,Auckland,09 366 1584,www.2degreesmobile.co.nz


<h5>Set Up Webpage Control</h5>

In [ ]:
options = Options()
options.page_load_strategy = 'eager'
# Add options you desire, e.g. headless

driver = wd.Chrome(
    'FULL DRIVER EXECUTABLE PATH\chromedriver.exe', options=options)

<h5>Set Up and Run Webpage URL Extraction and Testing Loop</h5>

In [ ]:
website_status_l = []

c = 0
for c in range(0, k):
    # extract website url from DataFrame
    if akl_cbd_biz['Web URL'].loc[c] == '':
        website_status = 'No URL'
    else:
        url = str("https://" + akl_cbd_biz['Web URL'].loc[c] + "/")
        print(url)        # To confirm correct URL
        driver.get(url)
        try:
            html = requests.get(url, verify=False, timeout=(60, 60)).text   # Timeout necessary to limit wait time
            test_soup = bs(html, 'html.parser')
            search = bool(test_soup.find_all(string=re.compile(
                'covid.+|.+contact tracing.+|.+qr.+', re.IGNORECASE)))      # key phrases are covid, contact tracing and QR
            if search == True:
                website_status = 'COVID Info Likely Present'
            else:
                website_status = 'No Obvious COVID Info'
        except:
            driver.execute_script("window.stop();")       # Occurs after 120 seconds or other access issues
            website_status = 'URL Parse Error'

    website_status_l.append(website_status)

    print(website_status)

    print(c)
    c += 1

driver.quit()

<h5>Append Data to Existing DataFrame, View and Save to Excel File</h5>

In [ ]:
akl_cbd_biz['Web URL COVID Info'] = website_status_l

akl_cbd_biz.head()

akl_cbd_biz.to_excel(
    r'FULL FILE PATH\FILE.xlsx', index=False, header=True)  # Saved as "url_covid_classifier.xlsx"

print("Saved to excel file")